# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4595089529293001648
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 10561234625670799465
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    t = Tokenizer()
    t.fit_on_texts(x)
    return t.texts_to_sequences(x), t
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if not length:
        length = max([len(sent) for sent in x])
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    inp = Input(shape=(input_shape[1:])) # reason not to use (input_shape[1],) is because GRU needs 3D input...
    rnn_out = GRU(64, return_sequences=True)(inp)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn_out)
    predictions = Activation('softmax')(logits)
    
    learning_rate = 0.001
    model = Model(inputs=inp, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# print(tmp_x.shape[1])
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 83us/step - loss: 3.4451 - acc: 0.4200 - val_loss: nan - val_acc: 0.4584
Epoch 2/10
110288/110288 [==============================] - 7s 64us/step - loss: 2.4252 - acc: 0.4714 - val_loss: nan - val_acc: 0.4862
Epoch 3/10
110288/110288 [==============================] - 7s 63us/step - loss: 2.1706 - acc: 0.5071 - val_loss: nan - val_acc: 0.5384
Epoch 4/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.9385 - acc: 0.5542 - val_loss: nan - val_acc: 0.5692
Epoch 5/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.7759 - acc: 0.5761 - val_loss: nan - val_acc: 0.5829
Epoch 6/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.6727 - acc: 0.5860 - val_loss: nan - val_acc: 0.5895
Epoch 7/10
110288/110288 [==============================] - 7s 64us/step - loss: 1.6015 - acc: 0.5934 - val_loss: nan - val_acc: 0.60

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inp = Input(shape=(input_shape[1],))
    embed = Embedding(output_dim=french_vocab_size, input_dim=200, input_length=input_shape[1])(inp)
    rnn_out = GRU(64, return_sequences=True, activation="tanh")(embed)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn_out)
    predictions = Activation('softmax')(logits)
    
    model = Model(inputs=inp, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
# print(tmp_x.shape[1])
# Train the neural network
embed_model_obj = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_model_obj.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_model_obj.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 83us/step - loss: 3.4384 - acc: 0.4305 - val_loss: nan - val_acc: 0.4921
Epoch 2/10
110288/110288 [==============================] - 9s 81us/step - loss: 1.9059 - acc: 0.5889 - val_loss: nan - val_acc: 0.6541
Epoch 3/10
110288/110288 [==============================] - 9s 81us/step - loss: 1.1997 - acc: 0.6952 - val_loss: nan - val_acc: 0.7408
Epoch 4/10
110288/110288 [==============================] - 9s 81us/step - loss: 0.8697 - acc: 0.7677 - val_loss: nan - val_acc: 0.7934
Epoch 5/10
110288/110288 [==============================] - 9s 81us/step - loss: 0.7039 - acc: 0.8067 - val_loss: nan - val_acc: 0.8199
Epoch 6/10
110288/110288 [==============================] - 9s 81us/step - loss: 0.6085 - acc: 0.8281 - val_loss: nan - val_acc: 0.8382
Epoch 7/10
110288/110288 [==============================] - 9s 81us/step - loss: 0.5440 - acc: 0.8435 - val_loss: nan - val_acc: 0.85

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

## my links for understanding Bidirectional
- [LSTM, Bidirectional](https://github.com/emoryjianghang/personality-prediction/blob/master/Understand%20LSTM.ipynb)
- [ai_course github](https://github.com/udacity/aind2-nlp-capstone)
- [attention model tutorial](https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/)

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inp = Input(shape=(input_shape[1:]))
    rnn_out = Bidirectional(GRU(64, return_sequences=True, activation="tanh", dropout=0.2), input_shape=input_shape[1:])(inp)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn_out)
    predictions = Activation('softmax')(logits)
    
    model = Model(inputs=inp, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2],1))
# print(tmp_x.shape[1])
# Train the neural network
embed_model_obj = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_model_obj.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_model_obj.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 102us/step - loss: 3.3707 - acc: 0.4544 - val_loss: nan - val_acc: 0.5075
Epoch 2/10
110288/110288 [==============================] - 11s 96us/step - loss: 2.1293 - acc: 0.5256 - val_loss: nan - val_acc: 0.5605
Epoch 3/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.7989 - acc: 0.5683 - val_loss: nan - val_acc: 0.5814
Epoch 4/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.6390 - acc: 0.5888 - val_loss: nan - val_acc: 0.5970
Epoch 5/10
110288/110288 [==============================] - 11s 98us/step - loss: 1.5456 - acc: 0.6002 - val_loss: nan - val_acc: 0.5977
Epoch 6/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.4837 - acc: 0.6099 - val_loss: nan - val_acc: 0.6009
Epoch 7/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.4360 - acc: 0.6176 - val_loss: nan - val_a

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

#### [why repeatVector in encoder-decoder](https://github.com/keras-team/keras/issues/5203)
- a better explanantion: [LSTM encoder-decoder](https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/)
    - basically, I misunderstood the output of LSTM() as encoder, the output is a context vector (1D), I thought it was 2D and did not get why repeat the vector many more times. Now it makes sense. Keep in mind, LSTM turns a 3D into 3D or 2D, either way, it process a whole batch at a time. So we do NOT need to wrap it with timedistributed like Dense() when encoutering 3D input. Dense takes a 2D and outputs a 2D. (or nD to nD without dimension reduction)
    - also, I am surprised that the same weights for the Dense() is used to output all tokens of the output sequence.

In [14]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    inp = Input(shape=(input_shape[1:])) # input: (max_input_length, 1)
    
    # encoder 
    rnn_encoded = GRU(256, dropout=0.1)(inp) # (max_input_length, 32)
    encode_logits = Dense(128, activation='relu')(rnn_encoded) # (max_input_length, 64)
    
    encoded = RepeatVector(output_sequence_length)(encode_logits) # (max_input_length, steps, 64)
    
    # decoder 
    decoded = GRU(256, dropout=0.1,return_sequences=True)(encoded) # output (max_input_length, steps, french_vocab_size)
    outputs = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoded) # output (steps, 32)


    model = Model(inputs=inp, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    return model
tests.test_encdec_model(encdec_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1,
    len(french_tokenizer.word_index) + 1)
encdec_rnn_model.summary()
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               198144    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 128)           0         
_________________________________________________________________
gru_10 (GRU)                 (None, 21, 256)           295680    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 21, 345)           88665     
Total params: 615,385
Trainable params: 615,385
Non-trainable params: 0
_________________________________________________________________
Trai

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [18]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inp = Input(shape=(input_shape[1],)) # (max_length,)
    embed = Embedding(output_dim=100, input_dim=english_vocab_size, input_length=input_shape[1])(inp) # (max_length, 100)
    
    # encoder
    bd_encoded = Bidirectional(GRU(512))(embed) # (maxlength, 128)
    dense_encoded = Dense(128, activation='relu')(bd_encoded) # (maxlength, 100)
    decoding_layer = RepeatVector(output_sequence_length)(dense_encoded)
    
    # decoder
    decoded_gru = Bidirectional(GRU(512, return_sequences=True))(decoding_layer)
    preds = TimeDistributed(Dense(french_vocab_size,activation='softmax'))(decoded_gru)
    
    model = Model(inputs=inp, outputs=preds)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(0.005),
                  metrics=['accuracy'])
    return model
tests.test_model_final(model_final)
print('Final Model Loaded')


Final Model Loaded


## Prediction (IMPLEMENTATION)

In [19]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(x.shape, y.shape[1], len(x_tk.word_index)+1, len(y_tk.word_index))
    model.fit(x,y,batch_size=1024, epochs=25,validation_split=0.2)


    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/25
110288/110288 [==============================] - 60s 544us/step - loss: 2.3552 - acc: 0.4932 - val_loss: nan - val_acc: 0.5974
Epoch 2/25
110288/110288 [==============================] - 58s 530us/step - loss: 1.1464 - acc: 0.6787 - val_loss: nan - val_acc: 0.7297
Epoch 3/25
110288/110288 [==============================] - 58s 530us/step - loss: 0.8058 - acc: 0.7600 - val_loss: nan - val_acc: 0.7982
Epoch 4/25
110288/110288 [==============================] - 59s 530us/step - loss: 0.5691 - acc: 0.8266 - val_loss: nan - val_acc: 0.8544
Epoch 5/25
110288/110288 [==============================] - 58s 530us/step - loss: 0.3964 - acc: 0.8804 - val_loss: nan - val_acc: 0.9116
Epoch 6/25
110288/110288 [==============================] - 59s 531us/step - loss: 0.2487 - acc: 0.9272 - val_loss: nan - val_acc: 0.9396
Epoch 7/25
110288/110288 [==============================] - 58s 530us/step - loss: 0.1797 - acc: 0.9479 - val_loss: nan -

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [21]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 348433 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

## Extra notes: please add 1 to french vocab size to avoid nan error in training because we want to handle oov as well... 

# Attention Layer

- [encoder-decoder vs. attention encoder-decoder](https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/): 95% vs 23%
- [How to build attention layer](https://medium.com/datalogue/attention-in-keras-1892773a4f22): this implementation is limited to N to N seq2seq; also, this attention layer handles decoding for us. I want to change it.